# Ejemplo de Estimación por Datos Panel

Referencia: Vella and M. Verbeek (1998), “Whose Wages Do Unions Raise? A Dynamic Model of Unionism and Wage Rate Determination for Young Men,” Journal of Applied Econometrics 13, 163-183.

## 1. Dependencias

In [1]:
!pip install linearmodels==4.24
!pip install linearmodels==4.5
!pip install linearmodels

  Using cached linearmodels-4.5-py2.py3-none-any.whl.metadata (5.8 kB)
Using cached linearmodels-4.5-py2.py3-none-any.whl (907 kB)
  Attempting uninstall: linearmodels
    Found existing installation: linearmodels 4.24
    Uninstalling linearmodels-4.24:
      Successfully uninstalled linearmodels-4.24


In [2]:
!pip install numpy==1.25.2
!pip install linearmodels==4.24

  Using cached linearmodels-4.24-cp311-cp311-linux_x86_64.whl
  Attempting uninstall: linearmodels
    Found existing installation: linearmodels 4.5
    Uninstalling linearmodels-4.5:
      Successfully uninstalled linearmodels-4.5


In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
from linearmodels.panel.model import PooledOLS, PanelOLS
from linearmodels.panel import RandomEffects
from linearmodels.panel import compare # Para comparar modelos

#
import warnings
warnings.filterwarnings('ignore')

## 2. Importación de datos

In [5]:
# Importamos el Data Set
data_to_load = 'wage_panel.csv'

# Read the and
wage_df = pd.read_csv(data_to_load)
wage_df.head()

,nr,year,black,exper,hisp,hours,married,educ,union,lwage,expersq,occupation
0,13,1980,0,1,0,2672,0,14,0,1.197540,1,9
1,13,1981,0,2,0,2320,0,14,1,1.853060,4,9
2,13,1982,0,3,0,2940,0,14,0,1.344462,9,9
3,13,1983,0,4,0,2960,0,14,0,1.433213,16,9
4,13,1984,0,5,0,3071,0,14,0,1.568125,25,5


### Los datos importados son:
* nr: person identifier
* year: 1980 to 1987
* black: =1 if black
* exper: labor market experience
* hisp: =1 if Hispanic
* hours: annual hours worked
* married: =1 if married
* educ: years of schooling
* union: =1 if in union
* lwage: log(wage)
* expersq: exper^2
* occupation: Occupation code

In [6]:
# Adecuaciones al índice para hacerlo Panel:
year = wage_df.year
wage_df = wage_df.set_index(['nr', 'year'])
wage_df#.head()

black  exper  hisp  hours  married  educ  union     lwage  \
nr    year                                                              
13    1980      0      1     0   2672        0    14      0  1.197540   
      1981      0      2     0   2320        0    14      1  1.853060   
      1982      0      3     0   2940        0    14      0  1.344462   
      1983      0      4     0   2960        0    14      0  1.433213   
      1984      0      5     0   3071        0    14      0  1.568125   
...           ...    ...   ...    ...      ...   ...    ...       ...   
12548 1983      0      8     0   2080        1     9      0  1.591879   
      1984      0      9     0   2080        1     9      1  1.212543   
      1985      0     10     0   2080        1     9      0  1.765962   
      1986      0     11     0   2080        1     9      1  1.745894   
      1987      0     12     0   3380        1     9      1  1.466543   

            expersq  occupation  
nr    year                       
13    1980        1           9  
      1981        4           9  
      1982        9           9  
      1983       16           9  
      1984       25           5  
...             ...         ...  
12548 1983       64           5  
      1984       81           5  
      1985      100           5  
      1986      121           5  
      1987      144           5  

[4360 rows x 10 columns]

In [7]:
# Agregamos una columna adicional de año (opción 1):
wage_df['year'] = pd.Categorical( year )
wage_df.head()

black  exper  hisp  hours  married  educ  union     lwage  expersq  \
nr year                                                                       
13 1980      0      1     0   2672        0    14      0  1.197540        1   
   1981      0      2     0   2320        0    14      1  1.853060        4   
   1982      0      3     0   2940        0    14      0  1.344462        9   
   1983      0      4     0   2960        0    14      0  1.433213       16   
   1984      0      5     0   3071        0    14      0  1.568125       25   

         occupation  year  
nr year                    
13 1980           9  1980  
   1981           9  1981  
   1982           9  1982  
   1983           9  1983  
   1984           5  1984

In [8]:
[ wage_df , pd.get_dummies(wage_df['year']) ]

[            black  exper  hisp  hours  married  educ  union     lwage  \
 nr    year                                                              
 13    1980      0      1     0   2672        0    14      0  1.197540   
       1981      0      2     0   2320        0    14      1  1.853060   
       1982      0      3     0   2940        0    14      0  1.344462   
       1983      0      4     0   2960        0    14      0  1.433213   
       1984      0      5     0   3071        0    14      0  1.568125   
 ...           ...    ...   ...    ...      ...   ...    ...       ...   
 12548 1983      0      8     0   2080        1     9      0  1.591879   
       1984      0      9     0   2080        1     9      1  1.212543   
       1985      0     10     0   2080        1     9      0  1.765962   
       1986      0     11     0   2080        1     9      1  1.745894   
       1987      0     12     0   3380        1     9      1  1.466543   
 
             expersq  occupation  ye

In [9]:
# Agregamos columnas de dummies de Year (opción 2):
wage_df = pd.concat( [ wage_df , pd.get_dummies(wage_df['year']) ], axis=1)

In [10]:
# Show data:
wage_df

black  exper  hisp  hours  married  educ  union     lwage  \
nr    year                                                              
13    1980      0      1     0   2672        0    14      0  1.197540   
      1981      0      2     0   2320        0    14      1  1.853060   
      1982      0      3     0   2940        0    14      0  1.344462   
      1983      0      4     0   2960        0    14      0  1.433213   
      1984      0      5     0   3071        0    14      0  1.568125   
...           ...    ...   ...    ...      ...   ...    ...       ...   
12548 1983      0      8     0   2080        1     9      0  1.591879   
      1984      0      9     0   2080        1     9      1  1.212543   
      1985      0     10     0   2080        1     9      0  1.765962   
      1986      0     11     0   2080        1     9      1  1.745894   
      1987      0     12     0   3380        1     9      1  1.466543   

            expersq  occupation  year   1980   1981   1982   1983   1984  \
nr    year                                                                 
13    1980        1           9  1980   True  False  False  False  False   
      1981        4           9  1981  False   True  False  False  False   
      1982        9           9  1982  False  False   True  False  False   
      1983       16           9  1983  False  False  False   True  False   
      1984       25           5  1984  False  False  False  False   True   
...             ...         ...   ...    ...    ...    ...    ...    ...   
12548 1983       64           5  1983  False  False  False   True  False   
      1984       81           5  1984  False  False  False  False   True   
      1985      100           5  1985  False  False  False  False  False   
      1986      121           5  1986  False  False  False  False  False   
      1987      144           5  1987  False  False  False  False  False   

             1985   1986   1987  
nr    year                       
13    1980  False  False  False  
      1981  False  False  False  
      1982  False  False  False  
      1983  False  False  False  
      1984  False  False  False  
...           ...    ...    ...  
12548 1983  False  False  False  
      1984  False  False  False  
      1985   True  False  False  
      1986  False   True  False  
      1987  False  False   True  

[4360 rows x 19 columns]

## 3. Regresión Pooled

In [11]:
# Definición de variables exógeneas y endógena
X = [ 'black','hisp','exper','expersq','married', 'educ', 'union', 'year', 'hours' ]
X = sm.add_constant( wage_df[X] )
X

const  black  hisp  exper  expersq  married  educ  union  year  \
nr    year                                                                   
13    1980    1.0      0     0      1        1        0    14      0  1980   
      1981    1.0      0     0      2        4        0    14      1  1981   
      1982    1.0      0     0      3        9        0    14      0  1982   
      1983    1.0      0     0      4       16        0    14      0  1983   
      1984    1.0      0     0      5       25        0    14      0  1984   
...           ...    ...   ...    ...      ...      ...   ...    ...   ...   
12548 1983    1.0      0     0      8       64        1     9      0  1983   
      1984    1.0      0     0      9       81        1     9      1  1984   
      1985    1.0      0     0     10      100        1     9      0  1985   
      1986    1.0      0     0     11      121        1     9      1  1986   
      1987    1.0      0     0     12      144        1     9      1  1987   

            hours  
nr    year         
13    1980   2672  
      1981   2320  
      1982   2940  
      1983   2960  
      1984   3071  
...           ...  
12548 1983   2080  
      1984   2080  
      1985   2080  
      1986   2080  
      1987   3380  

[4360 rows x 10 columns]

In [12]:
# Definición de variables exógeneas y endógena
X1 = [ 'black','hisp','exper','expersq','married', 'educ', 'union', 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 'hours' ]
X1 = sm.add_constant( wage_df[X1] )
X1

const  black  hisp  exper  expersq  married  educ  union   1981  \
nr    year                                                                    
13    1980    1.0      0     0      1        1        0    14      0  False   
      1981    1.0      0     0      2        4        0    14      1   True   
      1982    1.0      0     0      3        9        0    14      0  False   
      1983    1.0      0     0      4       16        0    14      0  False   
      1984    1.0      0     0      5       25        0    14      0  False   
...           ...    ...   ...    ...      ...      ...   ...    ...    ...   
12548 1983    1.0      0     0      8       64        1     9      0  False   
      1984    1.0      0     0      9       81        1     9      1  False   
      1985    1.0      0     0     10      100        1     9      0  False   
      1986    1.0      0     0     11      121        1     9      1  False   
      1987    1.0      0     0     12      144        1     9      1  False   

             1982   1983   1984   1985   1986   1987  hours  
nr    year                                                   
13    1980  False  False  False  False  False  False   2672  
      1981  False  False  False  False  False  False   2320  
      1982   True  False  False  False  False  False   2940  
      1983  False   True  False  False  False  False   2960  
      1984  False  False   True  False  False  False   3071  
...           ...    ...    ...    ...    ...    ...    ...  
12548 1983  False   True  False  False  False  False   2080  
      1984  False  False   True  False  False  False   2080  
      1985  False  False  False   True  False  False   2080  
      1986  False  False  False  False   True  False   2080  
      1987  False  False  False  False  False   True   3380  

[4360 rows x 16 columns]

In [13]:
#
Y = wage_df[ 'lwage' ]
Y

nr     year
13     1980    1.197540
       1981    1.853060
       1982    1.344462
       1983    1.433213
       1984    1.568125
                 ...   
12548  1983    1.591879
       1984    1.212543
       1985    1.765962
       1986    1.745894
       1987    1.466543
Name: lwage, Length: 4360, dtype: float64

In [14]:
# Regresión
model_1 = PooledOLS(Y, X1)
pooled_res_1 = model_1.fit()
print(pooled_res_1)

                          PooledOLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                        0.1952
Estimator:                  PooledOLS   R-squared (Between):              0.2034
No. Observations:                4360   R-squared (Within):               0.1856
Date:                Thu, May 08 2025   R-squared (Overall):              0.1952
Time:                        00:22:41   Log-likelihood                   -2966.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      70.221
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(15,4344)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             70.221
                            

In [15]:
# Regresión
model = PooledOLS(Y, X)
pooled_res = model.fit()
print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                        0.1952
Estimator:                  PooledOLS   R-squared (Between):              0.2034
No. Observations:                4360   R-squared (Within):               0.1856
Date:                Thu, May 08 2025   R-squared (Overall):              0.1952
Time:                        00:22:45   Log-likelihood                   -2966.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      70.221
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(15,4344)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             70.221
                            

## 4. Efectos aleatorios

In [16]:
# Regresión
model = RandomEffects(Y, X)
re_res = model.fit()
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:                  lwage   R-squared:                        0.1976
Estimator:              RandomEffects   R-squared (Between):              0.1716
No. Observations:                4360   R-squared (Within):               0.2013
Date:                Thu, May 08 2025   R-squared (Overall):              0.1854
Time:                        00:22:59   Log-likelihood                   -1569.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      71.315
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(15,4344)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             71.315
                            

In [17]:
# Descomposición de varianza
re_res.variance_decomposition

,Variance Decomposition
Effects,0.107522
Residual,0.120095
Percent due to Effects,0.472383


## 5. Efectos fijos

In [20]:
# Regresion con efectos fijos por entidad
# Omitimos: 'exper', 'black','hisp', 'educ', 'year' , 'const' # We should remove constant as well
X = [ 'expersq', 'union', 'married', 'hours' ]
X = sm.add_constant(wage_df[X]) # Remove this line as it adds the constant, which will likely get absorbed
# X = wage_df[X] # Use this line without a constant. sm.add_constant is called within the PanelOLS model if needed.
model = PanelOLS(Y, X, entity_effects = True)
fe_res = model.fit()
print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                        0.1454
Estimator:                   PanelOLS   R-squared (Between):             -0.0844
No. Observations:                4360   R-squared (Within):               0.1454
Date:                Thu, May 08 2025   R-squared (Overall):              0.0219
Time:                        00:23:44   Log-likelihood                   -1416.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      162.14
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                  F(4,3811)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             162.14
                            

In [19]:
# Regresion con efectos fijos por entidad
# Omitimos: 'exper', 'black','hisp', 'educ'
X = [ 'expersq', 'union', 'married', 'year', 'hours' ]
X = sm.add_constant(wage_df[X])
model = PanelOLS(Y, X, entity_effects = True)
fe_res = model.fit()
print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                        0.2022
Estimator:                   PanelOLS   R-squared (Between):             -0.0726
No. Observations:                4360   R-squared (Within):               0.2022
Date:                Thu, May 08 2025   R-squared (Overall):              0.0546
Time:                        00:23:33   Log-likelihood                   -1266.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      87.669
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(11,3804)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             87.669
                            

In [21]:
# Regresion con efectos fijos por entidad y tiempo
# Omitimos: 'exper', 'black','hisp', 'educ', 'year'
X = ['expersq', 'union', 'married', 'hours']
X = sm.add_constant(wage_df[X])
model = PanelOLS(Y, X, entity_effects = True, time_effects = True)
fe_te_res = model.fit()
print(fe_te_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                        0.0474
Estimator:                   PanelOLS   R-squared (Between):             -0.0726
No. Observations:                4360   R-squared (Within):              -0.6951
Date:                Thu, May 08 2025   R-squared (Overall):             -0.3606
Time:                        00:23:53   Log-likelihood                   -1266.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      47.359
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                  F(4,3804)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             47.359
                            

## 6. Comparación de modelos

In [23]:
!pip uninstall statsmodels linearmodels --yes
!pip install statsmodels==0.13.5 linearmodels==4.24

Found existing installation: statsmodels 0.14.4
Uninstalling statsmodels-0.14.4:
  Successfully uninstalled statsmodels-0.14.4
Found existing installation: linearmodels 4.24
Uninstalling linearmodels-4.24:
  Successfully uninstalled linearmodels-4.24
  Using cached linearmodels-4.24-cp311-cp311-linux_x86_64.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 62.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
plotnine 0.14.5 requires statsmodels>=0.14.0, but you have statsmodels 0.13.5 which is incompatible.


In [ ]:
from linearmodels.panel import compare
import pandas as pd

# This fixes the error in the compare function
def compare_patched(results, *, precision="%.4f", stars=True):
    """
    Compare the results of multiple models
    Parameters
    ----------
    results : dict[str, PanelResults]
        Dictionary containing model names and result instances.
    precision : str
        Precision to use when printing output. Has the form "%.Nf".
    stars : bool
        Indicates whether to use stars to mark the significance of parameters.
    Returns
    -------
    DataFrame
        DataFrame containing model comparison information.
    """
    if not isinstance(results, dict):
        raise TypeError("results must be a dictionary")
    vals = [v.summary.tables[1].iloc[1:] for v in results.values()]
    # This line was throwing the error so I fixed it.
    res = pd.concat(vals, axis=1, keys=results.keys())
    # ... rest of the function remains the same ...
    return res

In [22]:
#
print(compare( { 'Fix efect Ent.': fe_res,
                 'Fix Efect Ent-Time': fe_te_res,
                 'Radom efects': re_res,
                 'Pooled': pooled_res } ))

TypeError: concat() takes 1 positional argument but 2 were given

## 7. Varianza Robusta:

In [ ]:
# Rregresión
X = ['expersq', 'union', 'married', 'year', 'hours']
X = sm.add_constant(wage_df[X])
model = PanelOLS(Y, X, entity_effects = True)
#fe_res = model.fit(cov_type = 'robust')
# NOTAS: “unadjusted”, “homoskedastic” - Assume residual are homoskedastic, AND
#       “robust”, “heteroskedastic” - Control for heteroskedasticity using White’s estimator
fe_res = model.fit(cov_type = "clustered", cluster_entity = True)
# NOTAS: clust_entity_time = mod.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)

print(fe_res)